<a href="https://colab.research.google.com/github/carvalheirafc/deep_learning_stuff/blob/master/neural_networks/diabetes/task3/seq_model_2dot0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold


In [2]:
file_path = 'https://raw.githubusercontent.com/carvalheirafc/deep_learning_stuff/master/neural_networks/diabetes/diabetes_pca_result.csv'
df = pd.read_csv(file_path)
features = df.drop('Diagnóstico', axis = 1)
target = df['Diagnóstico']


print('Data Shape:', features.shape)
print('Labels/Target:', target.shape)
df.describe().transpose()

Data Shape: (768, 4)
Labels/Target: (768,)


,count,mean,std,min,25%,50%,75%,max
Component 1,768.0,-6.693141e-17,1.711960,-5.587340,-1.027072,0.152931,1.095689,5.963579
Component 2,768.0,-6.071532e-17,1.318184,-2.955806,-1.007096,-0.225966,0.959128,3.606582
Component 3,768.0,-3.324887e-17,1.015629,-3.203088,-0.646496,-0.103940,0.549774,4.754503
Component 4,768.0,2.349105e-18,0.966941,-2.192185,-0.664677,-0.115829,0.543242,4.125987
Diagnóstico,768.0,3.489583e-01,0.476951,0.000000,0.000000,0.000000,1.000000,1.000000


In [0]:
def get_train_test(X, y, n_splts=10):
  k_fold = StratifiedKFold(n_splits=n_splts, shuffle=True)

  X_train = []
  X_test = []
  y_train = []
  y_test = []
  
  for train_indices, test_indices in k_fold.split(X, y):
    X_train.append( X[train_indices] )
    y_train.append( y[train_indices] )
    X_test.append( X[test_indices] )
    y_test.append( y[test_indices] )
      

  return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test) 

In [0]:
x = tf.placeholder(dtype=tf.float32, shape=(None, 1))
y_true = tf.placeholder(dtype=tf.float32, shape=(None, 1))

In [0]:
def model():
  network = tf.keras.Sequential()

    
  network.add(tf.layers.Dense(units=32, activation='relu', input_shape=(features.shape[1],),
                              kernel_initializer='random_uniform', bias_initializer='ones'))

    
  network.add(tf.layers.Dense(units=32, activation='relu'))
  network.add(tf.layers.Dense(units=1, activation='sigmoid'))
  return network

In [0]:
def train(optimizer, seed, X, y):
  with tf.device('/gpu:0'):
    network = model()
    k_fold = StratifiedKFold(n_splits=25, shuffle=True, random_state=seed)
    cvscores = []
    prediction_acc = []
  
    network.compile(loss='mse', optimizer=optimizer, metrics=['accuracy', 'mse'])
    X_train = []
    for train, test in k_fold.split(X, y):
      X_train.append(X[train])
      history = network.fit(X[train], y[train], epochs=50, batch_size=10, verbose=0)
  
      scores = network.evaluate(X[test], y[test], verbose=2)
    
    
  return history

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def gpu(X, y, seed, optmizer):
  adam_model_history = train(optmizer, seed, X, y)

In [8]:
X = features.values
y = target.values
seed = 666
np.random.seed(seed)

adam_custom = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
gpu(X, y, seed, adam_custom)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
 - 0s - loss: 0.1664 - acc: 0.7419 - mean_squared_error: 0.1664
 - 0s - loss: 0.1772 - acc: 0.7419 - mean_squared_error: 0.1772
 - 0s - loss: 0.2049 - acc: 0.7419 - mean_squared_error: 0.2049
 - 0s - loss: 0.2174 - acc: 0.6452 - mean_squared_error: 0.2174
 - 0s - loss: 0.1652 - acc: 0.7419 - mean_squared_error: 0.1652
 - 0s - loss: 0.1399 - acc: 0.8065 - mean_squared_error: 0.1399
 - 0s - loss: 0.1520 - acc: 0.8065 - mean_squared_error: 0.1520
 - 0s - loss: 0.1903 - acc: 0.7097 - mean_squared_error: 0.1903
 - 0s - loss: 0.1379 - acc: 0.8065 - mean_squared_error: 0.1379
 - 0s - loss: 0.0774 - acc: 0.9032 - mean_squared_error: 0.0774
 - 0s - loss: 0.1167 - acc: 0.8387 - mean_squared_error: 0.1167
 - 0s - loss: 0.1505 - acc: 0.8065 - mean_squared_error: 0.1505
 - 0s - loss: 0.1017 - acc: 0.9032 - mean_squared_error: 0.1017
